In [1]:
#connexion avec une base de données PostgreSQL 
import psycopg2
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import re

try:
    conn = psycopg2.connect(
          user = "postgres",
          password = "efrei888",
          host = "localhost",
          port = "5432",
          database = "Beevora"
    )
    cur = conn.cursor()
    # Afficher la version de PostgreSQL 
    cur.execute("SELECT version();")
    version = cur.fetchone()
    print("Version : ", version,"\n")
    
    #fermeture de la connexion à la base de données
    #cur.close()
    #conn.close()
    #print("La connexion PostgreSQL est fermée")

except (Exception, psycopg2.Error) as error :
    print ("Erreur lors de la connexion à PostgreSQL", error)

Version :  ('PostgreSQL 13.4, compiled by Visual C++ build 1914, 64-bit',) 



In [2]:
cur.execute("select match, v_odds.date, v_odds.tournament ->> 'id' as tournament_id, tournaments.category as category, tournaments.name, label, field_name, field_label, \
	(winner -> 0) ->> 'id' as winner, odd, matches.teams[1] as team_1, \
	(select name from teams where id = matches.teams[1]) as name1, matches.teams[2] as team_2, \
	(select name from teams where id = matches.teams[2]) as name2, result #>'{Score,0}'->>'value' as final_score, \
	v_odds.bookmaker_id \
from v_odds, matches, tournaments \
where label = 'Résultat du match' \
and matches.id = v_odds.match \
and tournaments.id = (v_odds.tournament ->> 'id')::int \
and v_odds.sport = 1 \
and odd < 20 ")

res = cur.fetchall()
dfall = pd.DataFrame(res)
dfall.columns = ['match', 'date', 'tournament_id', 'category', 'tournament_name', 'label', 'field_name', 'field_label', 'winner', 'odd', 'team1', 'name1', 'team2', 'name2', 'final_score', 'bookmaker']

In [3]:
dfall

,match,date,tournament_id,category,tournament_name,label,field_name,field_label,winner,odd,team1,name1,team2,name2,final_score,bookmaker
0,28624054,2021-09-30 23:30:00+02:00,27665,49,Primera Division,Résultat du match,Home,Colo Colo,3155,1.27,3155.0,Colo Colo,7029.0,Nublense,0:1,8072
1,25822364,2021-09-29 14:00:00+02:00,704,270,Erovnuli Liga,Résultat du match,Home,FC Samgurali Tskaltubo,44217,1.81,44217.0,FC Samgurali Tskaltubo,5419.0,Torpedo Koutaissi,0:0,8072
2,25406638,2021-08-10 18:00:00+02:00,178,92,Premium Liiga,Résultat du match,Home,Flora Tallinn,3346,1.08,3346.0,Flora Tallinn,3343.0,Trans Narva,5:2,30238
3,28624054,2021-09-30 23:30:00+02:00,27665,49,Primera Division,Résultat du match,Home,Colo Colo,3155,1.35,3155.0,Colo Colo,7029.0,Nublense,0:1,-30239
4,28936218,2021-09-30 18:45:00+02:00,679,393,UEFA Ligue Europa,Résultat du match,Home,Fenerbahce,3052,2.45,3052.0,Fenerbahce,3245.0,Olympiakos,0:3,-8072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282193,27772134,2021-11-06 18:50:00+01:00,266,66,Première division,Résultat du match,Away,Beitar Jerusalem FC,5204,3.30,5333.0,Maccabi Petah Tikva FC,5204.0,Beitar Jerusalem FC,0:3,-8072
282194,27663880,2021-11-20 15:00:00+01:00,211,23,Superligue,Résultat du match,Away,Spartak Trnava,2406,1.56,7084.0,FC Vion Zlate Moravce - Vrable,2406.0,Spartak Trnava,0:0,6207
282195,25720750,2021-11-07 22:30:00+01:00,240,165,LigaPro Primera A,Résultat du match,Away,SC Delfin,241468,5.00,5257.0,LDU Quito,241468.0,SC Delfin,1:1,6207
282196,25720750,2021-11-07 22:30:00+01:00,240,165,LigaPro Primera A,Résultat du match,Away,SC Delfin,241468,4.90,5257.0,LDU Quito,241468.0,SC Delfin,1:1,9703


In [4]:
dfall['date'] = dfall['date'].map(pd.to_datetime)

def get_moy(dt): 
    return dt.month 

def get_dow(dt): 
    return dt.weekday() + 1

def get_dom(dt): 
    return dt.day 

def get_hour(dt): 
    return dt.hour

dfall['month'] = dfall['date'].map(get_moy)

dfall['day_of_week'] = dfall['date'].map(get_dow)

dfall['day_of_month'] = dfall['date'].map(get_dom)

dfall['hour'] = dfall['date'].map(get_hour)

In [5]:
l = [-30239, -30238, 29800, 8072]
dfall = dfall[~dfall.bookmaker.isin(l)]
dfall.reset_index(inplace=True)
dfall.drop(['date', 'tournament_name', 'label', 'name1', 'name2', 'field_label', 'final_score'], axis=1, inplace=True)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [6]:
dfall

,index,match,tournament_id,category,field_name,winner,odd,team1,team2,bookmaker,month,day_of_week,day_of_month,hour
0,2,25406638,178,92,Home,3346,1.08,3346.0,3343.0,30238,8,2,10,18
1,4,28936218,679,393,Home,3052,2.45,3052.0,3245.0,-8072,9,4,30,18
2,5,26251624,26,392,Home,None,3.72,6027.0,6019.0,5251,10,2,12,18
3,6,28624054,27665,49,Home,3155,1.36,3155.0,7029.0,8392,9,4,30,23
4,9,25720692,240,165,Home,21941,3.30,21941.0,5255.0,5251,10,1,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196144,282193,27772134,266,66,Away,5204,3.30,5333.0,5204.0,-8072,11,6,6,18
196145,282194,27663880,211,23,Away,2406,1.56,7084.0,2406.0,6207,11,6,20,15
196146,282195,25720750,240,165,Away,241468,5.00,5257.0,241468.0,6207,11,7,7,22
196147,282196,25720750,240,165,Away,241468,4.90,5257.0,241468.0,9703,11,7,7,22


In [7]:
for k in range(len(dfall)):
    if dfall['field_name'][k] == 'Draw':
        dfall['winner'][k] = 999999

<ipython-input-7-5389311611b1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfall['winner'][k] = 999999
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
dfall = dfall.dropna()
dfall.reset_index(inplace=True)
dfall.drop(['index'], axis=1, inplace=True)
dfall

D:\Anaconda\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,level_0,match,tournament_id,category,field_name,winner,odd,team1,team2,bookmaker,month,day_of_week,day_of_month,hour
0,0,25406638,178,92,Home,3346,1.08,3346.0,3343.0,30238,8,2,10,18
1,1,28936218,679,393,Home,3052,2.45,3052.0,3245.0,-8072,9,4,30,18
2,3,28624054,27665,49,Home,3155,1.36,3155.0,7029.0,8392,9,4,30,23
3,4,25720692,240,165,Home,21941,3.30,21941.0,5255.0,5251,10,1,4,2
4,5,28077158,238,44,Home,3026,2.21,3026.0,5136.0,5251,9,7,26,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192950,196144,27772134,266,66,Away,5204,3.30,5333.0,5204.0,-8072,11,6,6,18
192951,196145,27663880,211,23,Away,2406,1.56,7084.0,2406.0,6207,11,6,20,15
192952,196146,25720750,240,165,Away,241468,5.00,5257.0,241468.0,6207,11,7,7,22
192953,196147,25720750,240,165,Away,241468,4.90,5257.0,241468.0,9703,11,7,7,22


In [9]:
dfall['team1'] = dfall.team1.astype(int)
dfall['team2'] = dfall.team2.astype(int)
dfall['odd'] = dfall.odd.astype(np.float32)
dfall['match'] = dfall.match.astype(int)
dfall['bookmaker'] = dfall.bookmaker.astype(int)
dfall['tournament_id'] = dfall.tournament_id.astype(int)

<ipython-input-9-aa9191c1a982>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfall['team1'] = dfall.team1.astype(int)
<ipython-input-9-aa9191c1a982>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfall['team2'] = dfall.team2.astype(int)
<ipython-input-9-aa9191c1a982>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [10]:
dfall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192955 entries, 0 to 192954
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   level_0        192955 non-null  int64  
 1   match          192955 non-null  int32  
 2   tournament_id  192955 non-null  int32  
 3   category       192955 non-null  int64  
 4   field_name     192955 non-null  object 
 5   winner         192955 non-null  object 
 6   odd            192955 non-null  float32
 7   team1          192955 non-null  int32  
 8   team2          192955 non-null  int32  
 9   bookmaker      192955 non-null  int32  
 10  month          192955 non-null  int64  
 11  day_of_week    192955 non-null  int64  
 12  day_of_month   192955 non-null  int64  
 13  hour           192955 non-null  int64  
dtypes: float32(1), int32(5), int64(6), object(2)
memory usage: 16.2+ MB


In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [12]:
le.fit(dfall['field_name'])
le.classes_

array(['Away', 'Draw', 'Home'], dtype=object)

In [13]:
dfall['field_name'] = le.transform(dfall['field_name'])

<ipython-input-13-af85fd715525>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfall['field_name'] = le.transform(dfall['field_name'])


In [14]:
dfall['field_name'] = dfall.field_name.astype(int)
dfall['winner'] = dfall.winner.astype(int)

<ipython-input-14-02fabd5378bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfall['field_name'] = dfall.field_name.astype(int)
<ipython-input-14-02fabd5378bc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfall['winner'] = dfall.winner.astype(int)


In [15]:
dfall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192955 entries, 0 to 192954
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   level_0        192955 non-null  int64  
 1   match          192955 non-null  int32  
 2   tournament_id  192955 non-null  int32  
 3   category       192955 non-null  int64  
 4   field_name     192955 non-null  int32  
 5   winner         192955 non-null  int32  
 6   odd            192955 non-null  float32
 7   team1          192955 non-null  int32  
 8   team2          192955 non-null  int32  
 9   bookmaker      192955 non-null  int32  
 10  month          192955 non-null  int64  
 11  day_of_week    192955 non-null  int64  
 12  day_of_month   192955 non-null  int64  
 13  hour           192955 non-null  int64  
dtypes: float32(1), int32(7), int64(6)
memory usage: 14.7 MB


In [16]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

X = dfall[['match', 'tournament_id', 'category', 'field_name', 'winner', 'team1', 'team2', 'bookmaker', 'month', 'day_of_week',
       'day_of_month', 'hour']]
y = dfall['odd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=False)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((115773, 12), (77182, 12), (115773,), (77182,))

In [17]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

avg_odd = round(np.mean(y_train),2) #3.43
baseline_pred = np.repeat(avg_odd,y_test.shape[0])
baseline_rmse = np.sqrt(mean_squared_error(baseline_pred, y_test))

baseline_mae = mean_absolute_error(baseline_pred, y_test)

print("RMSE de référence sur les données de Validation :",baseline_rmse)
print("MAE de référence sur les données de Validation :",baseline_mae)

RMSE de référence sur les données de Validation : 2.6068196181584553
MAE de référence sur les données de Validation : 1.4898418065679373


# Linear regression 

In [18]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred = np.round(lm.predict(X_test), 2)
lm_rmse = np.sqrt(mean_squared_error(y_pred, y_test))
lm_train_rmse = np.sqrt(mean_squared_error(lm.predict(X_train), y_train))
lm_variance = abs(lm_train_rmse - lm_rmse)

lm_mae = mean_absolute_error(y_pred, y_test)

print("Test RMSE for Linear Regression is ",lm_rmse)
print("Train RMSE for Linear Regression is ",lm_train_rmse)
print("Variance for Linear Regression is ",lm_variance)
print("MAE for Linear Regression is ",lm_mae)

Test RMSE for Linear Regression is  3.3643308660127147
Train RMSE for Linear Regression is  1.7699287965023793
Variance for Linear Regression is  1.5944020695103354
MAE for Linear Regression is  2.198722369423744


# RandomForestRegressor

In [19]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100, random_state = 500, n_jobs=-1)
rf.fit(X_train,y_train)
rf_pred = np.round(rf.predict(X_test), 2)
rf_rmse = np.sqrt(mean_squared_error(rf_pred, y_test))
rf_train_rmse=np.sqrt(mean_squared_error(rf.predict(X_train), y_train))
rf_variance=abs(rf_train_rmse - rf_rmse)

rf_mae = mean_absolute_error(rf_pred, y_test)

print("Test RMSE for Random Forest is ",rf_rmse)
print("Train RMSE for Random Forest is ",rf_train_rmse)
print("Variance for Random Forest is ",rf_variance)
print("MAE for Random Forest is ",rf_mae)

Test RMSE for Random Forest is  3.1130700644064295
Train RMSE for Random Forest is  0.40840347514822495
Variance for Random Forest is  2.7046665892582045
MAE for Random Forest is  1.957055920586644


In [20]:
rf = RandomForestRegressor(n_estimators = 50, random_state = 500, n_jobs=-1)
rf.fit(X_train,y_train)
rf_pred = np.round(rf.predict(X_test), 2)
rf_rmse = np.sqrt(mean_squared_error(rf_pred, y_test))
rf_train_rmse=np.sqrt(mean_squared_error(rf.predict(X_train), y_train))
rf_variance=abs(rf_train_rmse - rf_rmse)

rf_mae = mean_absolute_error(rf_pred, y_test)

print("Test RMSE for Random Forest is ",rf_rmse)
print("Train RMSE for Random Forest is ",rf_train_rmse)
print("Variance for Random Forest is ",rf_variance)
print("MAE for Random Forest is ",rf_mae)

Test RMSE for Random Forest is  3.1202449787410256
Train RMSE for Random Forest is  0.41579166356957287
Variance for Random Forest is  2.7044533151714525
MAE for Random Forest is  1.9556658295977039


In [21]:
rf = RandomForestRegressor(n_estimators = 70, random_state = 500, n_jobs=-1)
rf.fit(X_train,y_train)
rf_pred = np.round(rf.predict(X_test), 2)
rf_rmse = np.sqrt(mean_squared_error(rf_pred, y_test))
rf_train_rmse=np.sqrt(mean_squared_error(rf.predict(X_train), y_train))
rf_variance=abs(rf_train_rmse - rf_rmse)

rf_mae = mean_absolute_error(rf_pred, y_test)

print("Test RMSE for Random Forest is ",rf_rmse)
print("Train RMSE for Random Forest is ",rf_train_rmse)
print("Variance for Random Forest is ",rf_variance)
print("MAE for Random Forest is ",rf_mae)

Test RMSE for Random Forest is  3.1070326783592983
Train RMSE for Random Forest is  0.4120893375413142
Variance for Random Forest is  2.694943340817984
MAE for Random Forest is  1.9456015661067307


In [22]:
rf = RandomForestRegressor(n_estimators = 40, random_state = 500, n_jobs=-1)
rf.fit(X_train,y_train)
rf_pred = np.round(rf.predict(X_test), 2)
rf_rmse = np.sqrt(mean_squared_error(rf_pred, y_test))
rf_train_rmse=np.sqrt(mean_squared_error(rf.predict(X_train), y_train))
rf_variance=abs(rf_train_rmse - rf_rmse)

rf_mae = mean_absolute_error(rf_pred, y_test)

print("Test RMSE for Random Forest is ",rf_rmse)
print("Train RMSE for Random Forest is ",rf_train_rmse)
print("Variance for Random Forest is ",rf_variance)
print("MAE for Random Forest is ",rf_mae)

Test RMSE for Random Forest is  3.1231387021741726
Train RMSE for Random Forest is  0.41939445806899595
Variance for Random Forest is  2.7037442441051764
MAE for Random Forest is  1.9567783946981205


In [23]:
rf = RandomForestRegressor(n_estimators = 20, random_state = 500, n_jobs=-1)
rf.fit(X_train,y_train)
rf_pred = np.round(rf.predict(X_test), 2)
rf_rmse = np.sqrt(mean_squared_error(rf_pred, y_test))
rf_train_rmse=np.sqrt(mean_squared_error(rf.predict(X_train), y_train))
rf_variance=abs(rf_train_rmse - rf_rmse)

rf_mae = mean_absolute_error(rf_pred, y_test)

print("Test RMSE for Random Forest is ",rf_rmse)
print("Train RMSE for Random Forest is ",rf_train_rmse)
print("Variance for Random Forest is ",rf_variance)
print("MAE for Random Forest is ",rf_mae)

Test RMSE for Random Forest is  3.1189695633966985
Train RMSE for Random Forest is  0.4367056316596896
Variance for Random Forest is  2.6822639317370087
MAE for Random Forest is  1.9526890993945538


# LightGBM

In [24]:
import lightgbm as lgb

train_data=lgb.Dataset(X_train,label=y_train)
param = {'num_leaves':30, 'num_trees':5000,'objective':'regression'}

param['metric'] = 'l2_root'
num_round=5000

cv_results = lgb.cv(param, train_data, 
                    num_boost_round=num_round, 
                    nfold=10,
                    verbose_eval=500, 
                    early_stopping_rounds=10,
                    stratified=False)

lgb_bst = lgb.train(param,train_data,len(cv_results['rmse-mean']))
lgb_pred = lgb_bst.predict(X_test)

lgb_train_rmse=np.sqrt(mean_squared_error(lgb_bst.predict(X_train), y_train))
lgb_rmse=np.sqrt(mean_squared_error(lgb_pred, y_test))
lgb_variance=abs(lgb_train_rmse - lgb_rmse)

lgb_mae = mean_absolute_error(lgb_pred, y_test)

print("Test RMSE for Light GBM is ",lgb_rmse)
print("Train RMSE for Light GBM is ",lgb_train_rmse)
print("Variance for lgb_train_rmse is ",lgb_variance)
print("MAE for LightGBM is ",lgb_mae)

D:\Anaconda\lib\site-packages\lightgbm\engine.py:573: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
D:\Anaconda\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001299 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1292
[LightGBM] [Info] Number of data points in the train set: 104193, number of used features: 12
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005762 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1292
[LightGBM] [Info] Number of data points in the train set: 104193, number of used features: 12
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005802 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1292
[LightGBM] [Info] Number of data points in the train set: 104193, number of used features: 12
[LightGBM] [Warning] Auto-choosing row-wise multi-threadi

D:\Anaconda\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[500]	cv_agg's rmse: 1.30497 + 0.0254389
[1000]	cv_agg's rmse: 1.21645 + 0.0247195
[1500]	cv_agg's rmse: 1.17254 + 0.022985
[2000]	cv_agg's rmse: 1.14874 + 0.023603
[2500]	cv_agg's rmse: 1.13528 + 0.0227014
[3000]	cv_agg's rmse: 1.12579 + 0.0223732


D:\Anaconda\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001548 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1292
[LightGBM] [Info] Number of data points in the train set: 115773, number of used features: 12
[LightGBM] [Info] Start training from score 3.097739
Test RMSE for Light GBM is  3.058684902674903
Train RMSE for Light GBM is  0.7226140591243143
Variance for lgb_train_rmse is  2.3360708435505884
MAE for LightGBM is  1.9601521444892114


# KNN neighbors

In [25]:
from sklearn import neighbors

knn = neighbors.KNeighborsRegressor(n_neighbors=100)
knn.fit(X_train, y_train)
y_pred = np.round(knn.predict(X_test), 2)

nb_rmse = np.sqrt(mean_squared_error(y_pred, y_test))
nb_train_rmse = np.sqrt(mean_squared_error(knn.predict(X_train), y_train))
nb_variance = abs(nb_train_rmse - nb_rmse)

knn_mae = mean_absolute_error(y_pred, y_test)

print("Test RMSE for KNN neighbors ",nb_rmse)
print("Train RMSE for KNN neighbors ",nb_train_rmse)
print("Variance for KNN neighbors is ",nb_variance)
print("MAE for KNN neighbors is ",knn_mae)

Test RMSE for Linear Regression is  2.766572
Train RMSE for Linear Regression is  1.6637182
Variance for Linear Regression is  1.1028538
MAE for knn is  1.6376319


In [26]:
from sklearn import neighbors

knn = neighbors.KNeighborsRegressor(n_neighbors=10)
knn.fit(X_train, y_train)
y_pred = np.round(knn.predict(X_test), 2)

nb_rmse = np.sqrt(mean_squared_error(y_pred, y_test))
nb_train_rmse = np.sqrt(mean_squared_error(knn.predict(X_train), y_train))
nb_variance = abs(nb_train_rmse - nb_rmse)

knn_mae = mean_absolute_error(y_pred, y_test)

print("Test RMSE for KNN neighbors is ",nb_rmse)
print("Train RMSE for KNN neighbors is ",nb_train_rmse)
print("Variance for KNN neighbors is ",nb_variance)
print("MAE for KNN neighbors is ",knn_mae)

Test RMSE for Linear Regression is  2.9363613
Train RMSE for Linear Regression is  1.4670458
Variance for Linear Regression is  1.4693155
MAE for knn is  1.7957895


In [27]:
from sklearn import neighbors

knn = neighbors.KNeighborsRegressor(n_neighbors=50)
knn.fit(X_train, y_train)
y_pred = np.round(knn.predict(X_test), 2)

nb_rmse = np.sqrt(mean_squared_error(y_pred, y_test))
nb_train_rmse = np.sqrt(mean_squared_error(knn.predict(X_train), y_train))
nb_variance = abs(nb_train_rmse - nb_rmse)

knn_mae = mean_absolute_error(y_pred, y_test)

print("Test RMSE for KNN neighbors is ",nb_rmse)
print("Train RMSE for KNN neighbors is ",nb_train_rmse)
print("Variance for KNN neighbors is ",nb_variance)
print("MAE for KNN neighbors is ",knn_mae)

Test RMSE for Linear Regression is  2.798751
Train RMSE for Linear Regression is  1.6253923
Variance for Linear Regression is  1.1733588
MAE for knn is  1.6727889


# Dummy regressor

In [28]:
from sklearn import dummy

dum = dummy.DummyRegressor(strategy='mean')
dum.fit(X_train, y_train)
y_pred = np.round(dum.predict(X_test), 2)

dum_rmse = np.sqrt(mean_squared_error(y_pred, y_test))
dum_train_rmse = np.sqrt(mean_squared_error(dum.predict(X_train), y_train))
dum_variance = abs(dum_train_rmse - dum_rmse)

dum_mae = mean_absolute_error(y_pred, y_test)

print("Test RMSE for Dummy regressor is ",dum_rmse)
print("Train RMSE for Dummy regressor is ",dum_train_rmse)
print("Variance for Dummy regressor is ",dum_variance)
print("MAE for Dummy regressor is ",dum_mae)

Test RMSE for Linear Regression is  2.6068196
Train RMSE for Linear Regression is  1.8529286
Variance for Linear Regression is  0.753891
MAE for dummyregressor is  1.4898417


In [29]:
dfall.corr()

,level_0,match,tournament_id,category,field_name,winner,odd,team1,team2,bookmaker,month,day_of_week,day_of_month,hour
level_0,1.000000,0.005957,0.000225,0.002121,-0.942423,0.004993,0.195833,0.005168,-0.000663,-0.005469,0.017108,0.001384,-0.000833,-0.004583
match,0.005957,1.000000,0.212708,0.279997,-0.001087,0.004408,-0.018453,0.062629,0.055641,-0.028014,0.214185,-0.061034,0.050130,0.086407
tournament_id,0.000225,0.212708,1.000000,0.331964,0.000381,-0.003102,-0.014966,-0.016575,-0.015353,0.006939,-0.067031,-0.121914,0.013461,-0.169339
category,0.002121,0.279997,0.331964,1.000000,0.001079,0.030919,0.035014,0.182719,0.187145,0.001142,-0.034578,-0.173147,0.004439,0.012482
field_name,-0.942423,-0.001087,0.000381,0.001079,1.000000,-0.003287,-0.205673,-0.003735,0.003575,0.002035,0.000027,-0.002133,-0.001067,0.001206
winner,0.004993,0.004408,-0.003102,0.030919,-0.003287,1.000000,0.113493,0.079844,0.077758,0.007291,-0.002438,-0.011870,-0.005782,-0.009436
odd,0.195833,-0.018453,-0.014966,0.035014,-0.205673,0.113493,1.000000,-0.015295,-0.004736,0.028624,0.014089,-0.031841,-0.026108,0.048066
team1,0.005168,0.062629,-0.016575,0.182719,-0.003735,0.079844,-0.015295,1.000000,0.210432,-0.003230,-0.014380,0.000109,0.005553,-0.103297
team2,-0.000663,0.055641,-0.015353,0.187145,0.003575,0.077758,-0.004736,0.210432,1.000000,-0.002254,-0.012106,-0.014678,0.002620,-0.092007
bookmaker,-0.005469,-0.028014,0.006939,0.001142,0.002035,0.007291,0.028624,-0.003230,-0.002254,1.000000,-0.111205,-0.003110,-0.022522,-0.000024


In [30]:
1.26 / dfall['odd'].mean()

0.3755975676666828